In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating new project at `~/Documents/eth_courses/notebooks/dynamics/julia/lagrangian`
  No Changes to `~/Documents/eth_courses/notebooks/dynamics/julia/lagrangian/Project.toml`
  No Changes to `~/Documents/eth_courses/notebooks/dynamics/julia/lagrangian/Manifest.toml`


In [8]:
using LinearAlgebra
using DifferentialEquations
using ForwardDiff
using GLMakie
using Rotations

In [16]:
# Rotation matrices 
B_R_P(ϕ) = RotZ(ϕ)
A_R_B(β) = RotX(π/2 - β) 

A_R_B (generic function with 1 method)

In [28]:
function kinematics(q)

    ϕ = q[1]
    ϕ̇ = q[1]

    #Parameters
    g = 9.81
    L=0.25
    β = π/6 
    M = 0.5
    m = 0.2
    Ω = 0.3


    # angular velocities

    # plate
    B_omega  = [0; 0 ;ϕ̇]
    A_omega  = [0; 0 ;Ω]

    I_omega_p = A_omega + A_R_B(β)  * B_omega
    I_omega_p

    # rod
    I_omega_rod1 = [0; 0 ;Ω]
    B_omega_rod2 = [0; 0 ;ϕ̇]
    I_omega_rod2 = I_omega_rod1 + A_R_B(β)  * B_omega_rod2

    # linear velocities 

    # plate
    I_v_ba = [L*Ω; 0 ;0]
    I_r_pb = A_R_B(β) *  B_R_P(ϕ) * [0; L/2; L/2] 
    I_v_po =  I_v_ba  + cross(I_omega_p,I_r_pb)

    # rods
    I_v_rod1 = [(L/2)*Ω; 0 ;0]
    I_v_rod2 = [Ω* (L + (L/2* cos(β))); 0; 0]
    
    r = [I_v_po;
         I_v_rod1;
         I_v_rod2;
         I_omega_p;
         I_omega_rod1;
         I_omega_rod2]
end

kinematics (generic function with 1 method)

In [31]:
size(kinematics([0,0]))

(18,)

In [29]:
function potential(q)
    
    r = kinematics(q)
    y1 = r[2]
    y2 = r[5]
    
    U = m*g*y1 + m*g*y2
end

potential (generic function with 1 method)

In [ ]:
function kinetic(q,q̇)
    
    K = ForwardDiff.jacobian(kinematics,q)
    ṙ = K*q̇
    
    T = 0.5*ṙ'*Diagonal([M; M; M;m;m;m;m;m;m; J; m; m; J])*ṙ
end

In [ ]:
function Lagrangian(q,q̇)
    return kinetic(q,q̇) - potential(q)
end

In [ ]:
function D1L(q,q̇)
    return ForwardDiff.gradient(dq->Lagrangian(dq,q̇),q)
end

function D2L(q,q̇)
    return ForwardDiff.gradient(dq̇->Lagrangian(q,dq̇),q̇)
end

function D2D2L(q,q̇)
    return ForwardDiff.jacobian(dq̇->D2L(q,dq̇),q̇)
end

function D1D2L(q,q̇)
    return ForwardDiff.jacobian(dq->D2L(dq,q̇),q)
end

In [ ]:
function dynamics(x,τ)
    q = x[1:2]
    q̇ = x[3:4]
    
    #Forced Euler-Lagrange Equation
    q̈ = D2D2L(q,q̇)\(D1L(q,q̇) - D1D2L(q,q̇)*q̇ + τ)
    
    ẋ = [q̇; q̈]
end

In [ ]:
function controller(x)
    q = x[1:2]
    q̇ = x[3:4]
    
    kp = 100.0
    kd = 10.0
    
    u = -kp*(q-[pi; 0.0]) - kd*q̇
end

In [ ]:
#Simulate
x0 = randn(4)

function f(x,p,t)
    #u = zeros(2)
    u = controller(x)
    dynamics(x, u)
end

tspan = (0.0,10.0)
prob = ODEProblem(f,x0,tspan)
sol = solve(prob,Tsit5(),abstol=1e-6,reltol=1e-6);

In [ ]:
plot(sol,idxs=(0,1))
plot!(sol,idxs=(0,2))

In [ ]:
function y(q)
    #We're going to apply a force in the x-direction at the end of the 2nd link
    return ℓ*sin(q[1]) + ℓ*sin(q[1]+q[2])
end

function Y(q)
    ForwardDiff.gradient(dq->y(dq),q)'
end

In [ ]:
function dynamics(x,F)
    q = x[1:2]
    q̇ = x[3:4]
    
    #damping constant
    c = 10.0
    
    #Forced Euler-Lagrange Equation
    q̈ = D2D2L(q,q̇)\(D1L(q,q̇) - D1D2L(q,q̇)*q̇ - c*q̇ + Y(q)'*F)
    
    ẋ = [q̇; q̈]
end

In [ ]:
#Simulate
x0 = zeros(4)

function f(x,p,t)
    F = 100.0
    dynamics(x, F)
end

tspan = (0.0,10.0)
prob = ODEProblem(f,x0,tspan)
sol = solve(prob,Tsit5(),abstol=1e-6,reltol=1e-6);

In [ ]:
plot(sol,idxs=(0,1))
plot!(sol,idxs=(0,2))